In [1]:
from ml_config import *
from jw_package import *
pd.options.mode.chained_assignment = None

import configuration completed !
train + key + weather merging started
train + key + weather merging finished
function configuration completed ! 
Good to go !


In [9]:
def missing_to_nan_weather(station_nbr=-1):
    print('missing_to_nan started !')
    
    full_weather = load_weather()
    
    if station_nbr != -1:
        full_weather = full_weather[full_weather['station_nbr']==station_nbr]
    
    # sunrise, sunset, category는 연속형변수가 아니므로 일단 뺌
    weathers = set(list(full_weather.columns)[2:20])
    targets = weathers - {'sunset','sunrise','codesum'}
    
    def trim_str(x):
        import re
                            # 실수만 추출
        result = re.findall('(\d+(?:\.\d+)?)',x)
        if result:
            return float(result[0])
        return np.nan
    
    for weather in targets:
        full_weather.loc[:,weather] = full_weather.loc[:,weather].astype(str).apply(trim_str).astype(float)
    
    print('missing_to_nan completed !')
    return full_weather.reset_index(drop=True)

In [3]:
def how_many_missing(nan_weather):
    
    weathers = set(list(nan_weather.columns)[2:20])
    targets = weathers - {'sunset','sunrise','codesum'}
    
    how_many = dict()
    total = len(nan_weather)
    
    for weather in targets:
        not_missing = len(nan_weather[weather].dropna())
    
        msg = 'total = '+ str(total) +' '
        msg += ', missing = '+ str(total - not_missing)
        
        how_many[weather] = msg
    
    return how_many

In [78]:
def all_missing_mean(nan_weather, weather):
    this_station_nbr = nan_weather.loc[0,'station_nbr']
    
    print('\t',weather,' ALL MISSING FEATURE')
    return nan_weather

In [79]:
def partial_missing_mean(nan_weather, weather):
    nan_feature = nan_weather[weather]

    nan_feature[nan_feature.isnull()] = nan_feature.mean()

    nan_weather.loc[:,weather] = nan_feature
    
    print('\t',weather,' : ',nan_feature.mean())
    
    if len(nan_feature[nan_feature.isnull()])==0:
        return nan_weather
    
    print('Error occured when partial_missing() by mean')
    print('station_nbr: ',nan_weather['station_nbr'][0],', feature = ',weather)
    return nan_weather

In [80]:
def replace_by_mean(nan_weather, weather):
    total = len(nan_weather)
    not_missing = len(nan_weather[weather].dropna())
    
    # 1. 아예 없다
    # 같은 station의 store의 평균
    if not_missing == 0:
        return all_missing_mean(nan_weather, weather)
        
    # 2. 있긴 있다
    # 그 store의 평균
    elif 0 < not_missing < total:
        return partial_missing_mean(nan_weather, weather)
    
    # missing이 없는 경우는 아무것도 안함
    return nan_weather

In [172]:
def filling_missing_by_mean(station_nbr):
    nan_weather = missing_to_nan_weather(station_nbr)
    how_many = how_many_missing(nan_weather)
    
    weathers = set(list(nan_weather.columns)[2:20])
    targets = weathers - {'sunset','sunrise','codesum'}
    
    for weather in targets:
        nan_weather = replace_by_mean(nan_weather,weather)
        
    return nan_weather, how_many

In [173]:
filled, how = filling_missing_by_mean(1)

missing_to_nan started !
missing_to_nan completed !
	 heat  :  15.006802721088436
	 resultdir  :  20.35111542192047
	 tavg  :  52.303206997084544
	 preciptotal  :  0.11841536614645913
	 dewpoint  :  39.1622934888241
	 resultspeed  :  5.3556741028128
	 tmin  :  42.59632139399806
	 wetbulb  :  46.1005859375
	 tmax  :  61.50338818973863
	 cool  :  2.3100097181729837
	 depart  ALL MISSING FEATURE
	 sealevel  :  30.013398247322314
	 avgspeed  :  6.5721899224806215
	 stnpressure  :  29.848089233753644
	 snowfall  ALL MISSING FEATURE


In [196]:
def filling_all_missing(weather, precision=10):

    this = weather.loc[0, 'station_nbr']
    others = [each for each in range(1, 21) if each != this]

    full_weather = missing_to_nan_weather()
#     no_nan = full_weather.dropna(axis=0, how='any').reset_index(drop=True)
    no_nan = full_weather.dropna(axis=0, how='any')
    
# 다른 station의 모든 행을 검사하면 시간이 너무 오래걸림
# precision이라는 변수와 pd.sample을 이용하여 그 중 일부만 추출
    no_nan = no_nan.sample(len(no_nan)//precision)
    nan = weather

    category = {'sunset', 'sunrise', 'codesum'}
    spare_columns = {each: weather[each] for each in list(category)}
    for each in list(category):
        weather.drop(each, axis=1, inplace=True)
        no_nan.drop(each, axis=1, inplace=True)

    # 독립변수끼리의 상관관계를 고려해 intact한 행만 고려한다.
    table = no_nan.describe()
    feature = list(weather.columns)[2:-3]
    
    print('\t','no_nan = ',len(no_nan))
    print('\t','nan = ',len(nan))

    for each in nan.index:
        if each%100==0:
            print('\t',each)
        nan_row = nan.loc[each, :]
        nan_z = pd.Series([(nan_row[f] - table[f]['mean']) /
                           table[f]['std'] for f in feature])
        # 얘가 nan이면 그 feature는 missing value임

        distance = dict()

        for idx in no_nan.index:
            intact = no_nan.loc[idx, :]
            intact_z = pd.Series(
                [(intact[f] - table[f]['mean']) / table[f]['std'] for f in feature])

            diff = abs(nan_z - intact_z)
            diff = diff.mean()

            distance[idx] = diff
            
        best_close_idx = min(distance, key=distance.get)
        missing_feature = nan_row.isnull()

        nan_row[missing_feature] = no_nan.loc[best_close_idx, missing_feature]
        weather.loc[each, :] = nan_row

    for each in list(category):
        weather[each] = spare_columns[each]

    return weather

In [212]:
def filling(station_nbr,precision=100):
    partial_filled, how = filling_missing_by_mean(station_nbr)
    
    ori = len(partial_filled.columns)
    fil = len(partial_filled.dropna(axis=1,how='all').columns)
    
    print('\t the number of total missing = ',ori - fil)
    
    if ori-fil != 0:
        all_filled = filling_all_missing(partial_filled,precision=100)
    
    else:
        all_filled = partial_filled
    
    return all_filled, how

In [187]:
filled,how = filling_missing_by_mean(1)

missing_to_nan started !
missing_to_nan completed !
	 heat  :  15.006802721088436
	 resultdir  :  20.35111542192047
	 tavg  :  52.303206997084544
	 preciptotal  :  0.11841536614645913
	 dewpoint  :  39.1622934888241
	 resultspeed  :  5.3556741028128
	 tmin  :  42.59632139399806
	 wetbulb  :  46.1005859375
	 tmax  :  61.50338818973863
	 cool  :  2.3100097181729837
	 depart  ALL MISSING FEATURE
	 sealevel  :  30.013398247322314
	 avgspeed  :  6.5721899224806215
	 stnpressure  :  29.848089233753644
	 snowfall  ALL MISSING FEATURE


In [219]:
for station_nbr in range(1,21):
    print('Station number = ',station_nbr)
    
    weather, how = filling(station_nbr,100)
    
    file_path = '../data/weather_partial_mean_all_best_close/{:02}.csv'.format(station_nbr)
    
    weather.to_csv(file_path)
    print('\t','write to csv completed !',file_path)
    print('\n')

Station number =  1
missing_to_nan started !
missing_to_nan completed !
	 heat  :  15.006802721088436
	 resultdir  :  20.35111542192047
	 tavg  :  52.303206997084544
	 preciptotal  :  0.11841536614645913
	 dewpoint  :  39.1622934888241
	 resultspeed  :  5.3556741028128
	 tmin  :  42.59632139399806
	 wetbulb  :  46.1005859375
	 tmax  :  61.50338818973863
	 cool  :  2.3100097181729837
	 depart  ALL MISSING FEATURE
	 sealevel  :  30.013398247322314
	 avgspeed  :  6.5721899224806215
	 stnpressure  :  29.848089233753644
	 snowfall  ALL MISSING FEATURE
	 the number of total missing =  2
missing_to_nan started !
missing_to_nan completed !
	 no_nan =  76
	 nan =  1035
	 0
	 100
	 200
	 300
	 400
	 500
	 600
	 700
	 800
	 900
	 1000
	 write to csv completed ! ../data/weather_partial_mean_all_best_close/01.csv


Station number =  2
missing_to_nan started !
missing_to_nan completed !
	 heat  :  16.619745845552295
	 resultdir  :  22.662135922330094
	 tavg  :  50.01270772238514
	 preciptotal  :  0.

PermissionError: [Errno 13] Permission denied: '../data/weather_partial_mean_all_best_close/02.csv'

In [220]:
for station_nbr in range(2,21):
    print('Station number = ',station_nbr)
    
    weather, how = filling(station_nbr,100)
    
    file_path = '../data/weather_partial_mean_all_best_close/{:02}.csv'.format(station_nbr)
    
    weather.to_csv(file_path)
    print('\t','write to csv completed !',file_path)
    print('\n')

Station number =  2
missing_to_nan started !
missing_to_nan completed !
	 heat  :  16.619745845552295
	 resultdir  :  22.662135922330094
	 tavg  :  50.01270772238514
	 preciptotal  :  0.14992152466367759
	 dewpoint  :  38.70553935860058
	 resultspeed  :  8.236699029126209
	 tmin  :  41.50773694390716
	 wetbulb  :  44.66143277723259
	 tmax  :  57.823984526112184
	 cool  :  1.632453567937439
	 depart  :  5.950604838709666
	 sealevel  :  30.00888671874994
	 avgspeed  :  9.721435499515033
	 stnpressure  :  28.905160349854242
	 snowfall  :  0.22693110647181655
	 the number of total missing =  0
	 write to csv completed ! ../data/weather_partial_mean_all_best_close/02.csv


Station number =  3
missing_to_nan started !
missing_to_nan completed !
	 heat  :  8.941634241245136
	 tavg  :  62.66147859922178
	 preciptotal  :  0.09329386437029079
	 tmin  :  51.171345595353344
	 wetbulb  :  54.674418604651166
	 tmax  :  73.675701839303
	 cool  :  6.603112840466926
	 depart  :  7.060180541624863
	 sea

d:\anaconda\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars
d:\anaconda\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars


	 100
	 200
	 300
	 400
	 500
	 600
	 700
	 800
	 900
	 1000
	 write to csv completed ! ../data/weather_partial_mean_all_best_close/09.csv


Station number =  10
missing_to_nan started !
missing_to_nan completed !
	 heat  :  1.9980525803310616
	 tavg  :  73.11295034079845
	 preciptotal  :  0.17476624857468606
	 wetbulb  :  65.03391472868216
	 cool  :  10.111002921129502
	 depart  ALL MISSING FEATURE
	 sealevel  :  30.049922555663073
	 avgspeed  :  8.401740812379106
	 stnpressure  :  30.024148936170207
	 snowfall  ALL MISSING FEATURE
	 the number of total missing =  2
missing_to_nan started !
missing_to_nan completed !
	 no_nan =  76
	 nan =  1035
	 0
	 100
	 200
	 300
	 400
	 500
	 600
	 700
	 800
	 900
	 1000
	 write to csv completed ! ../data/weather_partial_mean_all_best_close/10.csv


Station number =  11
missing_to_nan started !
missing_to_nan completed !
	 heat  :  2.9980601357904946
	 resultdir  :  17.088205128205132
	 tavg  :  69.70126091173618
	 preciptotal  :  0.1592927094668